In [ ]:
# install required packages 
#!pip install beautifulsoup4 lxml selenium webdriver-manager

In [82]:
# import necessary modules
import time
import requests
from csv import writer
from bs4 import BeautifulSoup
from lxml import etree as et
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [83]:
# define job and location search keywords
job_search_keyword = ['Data+Scientist']#, 'Data+Analyst', 'Product+Analyst', 'BI+Analyst']
location_search_keyword = ['New+York']#, 'Los+Angeles', 'Chicago']

# define base and pagination URLs
base_url = 'https://www.indeed.com'
paginaton_url = "https://www.indeed.com/jobs?q={}&l={}&radius=35&start={}"

# check also Sweden and France 
# Just pick the 3 largest cities in each country 

In [84]:
# Function to get DOM from given URL
def get_dom(url):
    driver.get(url)
    time.sleep(3)  # Ensure page loads
    page_content = driver.page_source
    product_soup = BeautifulSoup(page_content, 'html.parser')
    dom = et.HTML(str(product_soup))
    return dom

# Function to extract job link
def get_job_link(job):
    try:
        return job.xpath('./descendant::h2/a/@href')[0]
    except Exception:
        return 'Not available'

# Function to extract job description
def get_job_desc(job_link):
    job_dom = get_dom(job_link)
    try:
        job_desc = job_dom.xpath('//*[@id="jobDescriptionText"]//text()')
        return " ".join(job_desc).strip() if job_desc else 'Not available'
    except Exception:
        return 'Not available'

def get_skills(job_link):
    job_dom = get_dom(job_link)
    try:
        # Use a refined XPath to get all relevant text within the profile insights
        skills = job_dom.xpath('//div[contains(@class, "profile-insights")]//text()')
        return " ".join(skills).strip() if skills else 'Not available'
    except Exception:
        return 'Not available'

# functions to extract job title
def get_job_title(job):
   try:
       job_title = job.xpath('./descendant::h2/a/span/text()')[0]
   except Exception as e:
       job_title = 'Not available'
   return job_title

# functions to extract the company name
def get_company_name(job):
   try:
       company_name = job.xpath('.//span[@data-testid="company-name"]/text()')[0]
       #company_name = job.xpath('./descendant::span[@class="companyName"]/text()')[0]
   except Exception as e:
       company_name = 'Not available'
   return company_name

# functions to extract the company location
def get_company_location(job):
   try:
       company_location = job.xpath('.//div[@data-testid="text-location"]/text()')[0]
       #company_location = job.xpath('./descendant::div[@class="companyLocation"]/text()')[0]
   except Exception as e:
       company_location = 'Not available'
   return company_location

# functions to extract salary information
'''
def get_salary(job):
   try:
       salary = job.xpath('./descendant::span[@class="estimated-salary"]/span/text()')
   except Exception as e:
       salary = 'Not available'
   if len(salary) == 0:
       try:
           salary = job.xpath('./descendant::div[@class="metadata salary-snippet-container"]/div/text()')[0]
       except Exception as e:
           salary = 'Not available'
   else:
       salary = salary[0]
   return salary
'''

def get_salary(job_link):
    job_dom = get_dom(job_link)
    try:
        # Use the provided XPath to get the salary text
        salary = job_dom.xpath('//*[@id="salaryInfoAndJobType"]//text()')
        return " ".join(salary).strip() if salary else 'Not available'
    except Exception:
        return 'Not available'
        
# functions to extract job type
def get_job_type(job):
   try:
       job_type = job.xpath('./descendant::div[@class="metadata"]/div/text()')[0]
   except Exception as e:
       job_type = 'Not available'
   return job_type

def get_total_pages(job_keyword, location_keyword):

    # URL of Indeed job search
    url = f"https://www.indeed.com/jobs?q={job_keyword}&l={location_keyword}"
    driver.get(url)

    try:
        # Wait for the element containing the job count to appear
        job_count_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "jobsearch-JobCountAndSortPane-jobCount")]'))
        )
        
        # Extract the text from the element
        job_count_text = job_count_element.text
        print(f"Job count text: {job_count_text}")
        job_count = int(job_count_text.split('+')[0].replace(',', '').strip())  # Handle commas and extra spaces

        # Each page shows 15 jobs
        jobs_per_page = 15

        # Calculate the total number of pages
        total_pages = math.ceil(job_count / jobs_per_page)
        print(f"Total pages: {total_pages}")

        return total_pages

    except Exception as e:
        print(f"Error extracting job count: {e}")
        return 0  # Return 0 if there's an error

    finally:
        # Close the browser
        #driver.quit()
        pass

In [81]:
#inspect content of a Python file 
#import inspect
#lines = inspect.getsource(get_total_pages)
#print(lines)

def get_total_pages(job_keyword, location_keyword):

    # URL of Indeed job search
    url = f"https://www.indeed.com/jobs?q={job_keyword}&l={location_keyword}"
    driver.get(url)

    try:
        # Wait for the element containing the job count to appear
        job_count_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "jobsearch-JobCountAndSortPane-jobCount")]'))
        )
        
        # Extract the text from the element
        job_count_text = job_count_element.text
        print(f"Job count text: {job_count_text}")
        job_count = int(job_count_text.split('+')[0].replace(',', '').strip())  # Handle commas and extra spaces

        # Each page shows 15 jobs
        jobs_per_page = 15

        # Calculate the total number of pages
        total_pages = math.ceil(job_count / jobs_per_page)
        print(f"Total pages: {total_pages}")

        return total_pages

    except Exception as e:
        print

In [ ]:
# Initialize Chrome webdriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open URL
driver.get("https://www.indeed.com/q-USA-jobs.html")

# Open a CSV file to write the job listings data
with open('indeed_jobs1.csv', 'w', newline='', encoding='utf-8') as f:
    theWriter = writer(f)
    heading = ['page', 'job_link', 'search_keyword', 'search_location', 'job_title', 'company_name', 'company_location', 'salary', 'job_description']
    theWriter.writerow(heading)

    for job_keyword in job_search_keywords:
        for location_keyword in location_search_keywords:
            print(f"Searching for: {job_keyword} in {location_keyword}")
            all_jobs = []

            # Find total number of pages
            total_pages = get_total_pages(job_keyword, location_keyword)
            print(f"Total pages found: {total_pages}")  # Print total pages

            for page_no in range(total_pages):  # Modify range to get more pages if needed
                print(f"Fetching Page number: {page_no + 1}")  # Display page number (1-based)
                url = f"https://www.indeed.com/jobs?q={job_keyword}&l={location_keyword}&start={page_no * 10}"
                page_dom = get_dom(url)
                
                # Extract jobs from current page
                jobs = page_dom.xpath('//div[@class="job_seen_beacon"]')
                print(f"Jobs found on page {page_no + 1}: {len(jobs)}")  # Print number of jobs found
                
                # Process each job
                for job in jobs:
                    job_link = base_url + get_job_link(job)
                    job_title = get_job_title(job)
                    company_name = get_company_name(job)
                    company_location = get_company_location(job)
                    salary = get_salary(job_link)
                    job_desc = get_job_desc(job_link)  # Extract job description for this job

                    record = [page_no + 1, job_link, job_keyword, location_keyword, job_title, company_name, company_location, salary, job_desc]

                    print(page_no + 1, job_link)  # Print page number (1-based index) and job link
                    theWriter.writerow(record)  # Write the record to CSV

# Closing the web browser
driver.quit()

Searching for: Data Scientist in New York
Job count text: 300+ jobs
Total pages: 20
Total pages found: 20
Fetching Page number: 1
Jobs found on page 1: 15
1 https://www.indeed.com/rc/clk?jk=d1b96df5f19292fd&bb=uFHC_idxkOdMhSd7jLvpbdmVqkVMRizgaSppHa9RnBr5gnqggwT93h0dkyJfb49X6Z38zSWR6IU_au0MIH0dA2STlyJXalgYBNnd73aMRl6PPOcqUyRb5Q%3D%3D&xkcb=SoCu67M37HRhBYygNx0LbzkdCdPP&fccid=a5b4499d9e91a5c6&vjs=3
1 https://www.indeed.com/rc/clk?jk=99eba59d82a9848e&bb=uFHC_idxkOdMhSd7jLvpbdhn7tsseqUWJpd0uRnCwxvyFHxwr2_SZq8Oiv73i9xgXGeIFo2L4AvwsqqRyCpZTVUo7IdYw52m9iAX7or8z1HnkWUwrUbAIvxAkAJ83tTh&xkcb=SoAa67M37HRhBYygNx0KbzkdCdPP&fccid=7d7b563c6a3a9653&vjs=3
1 https://www.indeed.com/rc/clk?jk=be8ea87cdf172e4a&bb=uFHC_idxkOdMhSd7jLvpbQwud5KikuDEe9aAleb4Sbrz0uB5BZATDRCb2Pjap6FijFBpvMD-T1GeAN4uNVXBMeuHUVR3a4T1Rn1TofLABUKS5zV9L_5l9g%3D%3D&xkcb=SoCH67M37HRhBYygNx0JbzkdCdPP&fccid=a5b4499d9e91a5c6&vjs=3
1 https://www.indeed.com/rc/clk?jk=0d47ada5c09e40f9&bb=uFHC_idxkOdMhSd7jLvpbchWD--rU99SyqviGfVnqKtFkxcESA2Tkl1LM

In [ ]:
'''
# for inspection and debugging: inspect the DOM
# or inspect element using Developer tools

# initialize Chrome webdriver using ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# open URL 
driver.get("https://www.indeed.com/q-USA-jobs.html?vjk=823cd7ee3c203ac3")
with open('indeed_jobs1_dom.txt', 'w', encoding='utf-8') as f:
    url = paginaton_url.format(job_search_keyword[0], location_search_keyword[0], page_no)
    time.sleep(3)
    page_dom = get_dom(url)  # Get the DOM of the page

    # Convert the DOM object to a string with pretty formatting
    dom_html = etree.tostring(page_dom, pretty_print=True, encoding='unicode')

    # Write the HTML string to the file
    f.write(dom_html)

# Save the full page DOM to a file for manual inspection
with open('indeed_full_page.html', 'w', encoding='utf-8') as f:
    time.sleep(4)
    page_source = driver.page_source
    f.write(page_source)
'''

In [ ]:
# ideas for visualization 
# number of listings per location
# average salary for each position 
# average salary by job AND location https://www.indeed.com/rc/clk?jk=9e205f9634a5bfdb&bb=Kaksi2QKb_5G01a0IYYWte3dB7Sd8vTC2PsDgK8Wcrl-vt-ov1bmllcNU4eq-ARapdqrzuLgp5TwUhCEMRJS0x0bX5YFJls_0xhvnMGs1P2Xvjh_c0znjkjJ1X6wishm&xkcb=SoBo67M37Ey6rjTbpx0LbzkdCdPP&fccid=2e5243142d98319d&vjs=3


# keyword extraction
# frequency differences for different job titles 
# tech stack keywords 

In [79]:
%history

# import necessary modules
import time
import requests
from csv import writer
from bs4 import BeautifulSoup
from lxml import etree as et
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# define job and location search keywords
job_search_keyword = ['Data+Scientist']#, 'Data+Analyst', 'Product+Analyst', 'BI+Analyst']
location_search_keyword = ['New+York']#, 'Los+Angeles', 'Chicago']

# define base and pagination URLs
base_url = 'https://www.indeed.com'
paginaton_url = "https://www.indeed.com/jobs?q={}&l={}&radius=35&start={}"

# check also Sweden and France 
# Just pick the 3 largest cities in each country
# Function to get DOM from given URL
def get_dom(url):
    driver.get(url)
    time.sleep(3)  # Ensure page loads
    page_content = driver.page_source
    product_soup = BeautifulSoup(page_content, 'html.parser')
    dom = et.HTML(str(product_soup))
    return dom

# Function to extract job 